# EnergyPlus Utility

In [11]:
!pip install -q "energy-plus-utility @ git+https://github.com/janithcyapa/energy-plus-utility.git@main"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [10]:
import importlib.metadata
ver = importlib.metadata.version("energy-plus-utility")
print(f"\n✅ Installed 'energy-plus-utility' version: {ver}")

# from eplus import prepare_colab_eplus
# prepare_colab_eplus()


✅ Installed 'energy-plus-utility' version: 0.1.0


In [ ]:
# In your main script or core.py
sim = BaseEPlusUtil(...)
sim.set_model(...)

# OLD WAY (Composition):
# handle = sim.api.exchange.get_variable_handle(sim.state, "Zone Mean Air Temperature", "Zone 1")

# NEW WAY (Inheritance):
# You can access .exchange directly because sim IS an EnergyPlusAPI instance
handle = sim.exchange.get_variable_handle(sim.state, "Zone Mean Air Temperature", "Zone 1")